In [43]:
import numpy as np
import altair as alt 
import pandas as pd

In [44]:
def init_parameters(n_features, n_neurons, n_output): 
    """ TODO: mejorar explicación
    Parametros inciales de la red neuronal con valores aleatorios
        W1: Pesos de la capa de entrada 
        W2: Pesos de la capa a la de salida 
        b1: bias para la capa
        b2: bias para la capa de salida
    """

    np.random.seed(100)
    W1 = np.random.uniform(size = (n_features, n_neurons))
    b1 = np.random.uniform(size = (1, n_neurons))

    W2 = np.random.uniform(size = (n_neurons, n_output))
    b2 = np.random.uniform(size = (1, n_output))

    return {
        "W1" : W1 
        , "b1" : b1 
        , "W2" : W2
        , "b2" : b2 
    }

In [45]:
def linear_function(W, X, b): 
    """
    Calcula la transformación lineal XW + b
    """
    return (X @ W)+ b 

In [46]:
def sigmoid_func(Z): 
    """
    Función de activación sigmoide para una Zn
    """
    return 1 / (1 + np.exp(-Z))

In [47]:
def cost_function(A, y): 
    """
    Calcula la función de costo Binary Cross Entropy entre las predicciones A y los valores reales y
    """

    return -np.mean(y * np.log(A) + (1 - y) * np.log(1 - A))
    # https://www.geeksforgeeks.org/deep-learning/binary-cross-entropy-log-loss-for-binary-classification/

In [48]:
def predict(X, W1, W2, b1, b2): 
    """
    Predicción de la red neuronal
        1. Calcula la transformación lineal + activación para la capa oculta
        2. Calcula la transformación lineal + activación para la capa de salida
        3. Aplica umbral de 0.5 para obtener predicciones binarias

    """
    Z1 = linear_function(W1, X, b1)
    S1 = sigmoid_func(Z1) # a1
    Z2 = linear_function(W2, S1, b2)
    S2 = sigmoid_func(Z2) # a2
    return np.where(S2 >= 0.5, 1, 0)

In [49]:
def fit(X, y, n_features = 2, n_neurons = 3, n_output = 1, iterations = 10, eta = 0.001): 
    """
    Entrenamiento de la red neuronal 
       1. Calcula las activaciones
       2. Calcula el error
       3. Calcula gradientes y actualiza pesos
    """
    params = init_parameters(
        n_features= n_features
        , n_neurons= n_neurons
        , n_output= n_output
    )

    errors = [] 

    for _ in range(iterations): 
        """ Activaciones """
        Z1 = linear_function(params['W1'], X, params['b1'])
        S1 = sigmoid_func(Z1)
        Z2 = linear_function(params['W2'], S1, params['b2'])
        S2 = sigmoid_func(Z2)
        """ Cálculo de errores """
        error = cost_function(S2, y)
        errors.append(error)

        """ Claclulo de gradientes y nnuevos pesos """
        delta2 = (S2 - y) # derivada
        W2_gradients = S1.T @ delta2 
        params["W2"] = params["W2"] - W2_gradients * eta

        params["b2"] = params["b2"] - np.sum(delta2, axis = 0, keepdims= True) * eta 

        delta1 = (delta2 @ params["W2"].T) * S1 * (1 - S1)
        W1_gradients = X.T @ delta1 
        params["W1"] = params["W1"] - W1_gradients * eta 

        params["b1"] = params["b1"] - np.sum(delta1, axis = 0, keepdims= True) * eta 

    return errors, params 


        

In [50]:
y = np.array([[0, 1, 1, 0]]).T 
X = np.array([[0, 0, 1, 1]
              ,[0, 1, 0, 1]]).T 

In [51]:
errors, params = fit(X, y, iterations=5000, eta = 0.1)

In [52]:
y_pred = predict(X, params["W1"], params["W2"], params["b1"], params["b2"])
num_correct_predictions = (y_pred == y).sum()
accuracy = (num_correct_predictions / y.shape[0]) * 100
print('Multi-layer perceptron accuracy: %.2f%%' % accuracy)


Multi-layer perceptron accuracy: 100.00%


In [53]:
alt.data_transformers.disable_max_rows()
df = pd.DataFrame({"errors":errors, "time-step": np.arange(0, len(errors))})
alt.Chart(df).mark_line().encode(x="time-step", y="errors").properties(title='Chart 2')


alt.Chart(...)

# Mismo Dataset en Framework Pytorch

In [54]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
import altair as alt

In [55]:
# Inputs: XOR problem
X = np.array([[0, 0, 1, 1],
              [0, 1, 0, 1]]).T  # Shape (4, 2)
y = np.array([[0, 1, 1, 0]]).T  # Shape (4, 1)

# Convert to torch tensors
X_tensor = torch.tensor(X, dtype=torch.float32)
y_tensor = torch.tensor(y, dtype=torch.float32)

# Define the model
class SimpleNet(nn.Module):
    def __init__(self):
        super(SimpleNet, self).__init__()
        self.fc1 = nn.Linear(2, 16)  
        self.fc2 = nn.Linear(16, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.sigmoid(self.fc1(x))
        x = self.sigmoid(self.fc2(x))
        return x

model = SimpleNet()

# Loss and optimizer
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters())

# Training loop
errors = []
epochs = 5000

for epoch in range(epochs):
    optimizer.zero_grad()
    outputs = model(X_tensor)
    loss = criterion(outputs, y_tensor)
    loss.backward()
    optimizer.step()
    errors.append(loss.item())

# Convert errors to DataFrame and plot
df2 = pd.DataFrame({"errors": errors, "time-step": np.arange(epochs)})

alt.Chart(df2).mark_line().encode(
    x="time-step",
    y="errors"
).properties(title='Chart 3')


alt.Chart(...)